In [11]:
from dotenv import load_dotenv
load_dotenv()

load_dotenv(override=True)

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-922cd1b8-ffad-4c01-ab10-fa7bb8d61c16-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
import os
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")
PINECONE_INDEX_NAME_2 = os.getenv("PINECONE_INDEX_NAME_2")
PINECONE_NAMESPACE_2 = os.getenv("PINECONE_NAMESPACE_2")

print(PINECONE_API_KEY, PINECONE_INDEX_NAME, PINECONE_NAMESPACE, PINECONE_INDEX_NAME_2, PINECONE_NAMESPACE_2)

pcsk_5TkRik_EzGTnXtpN1c26rjgbrhySAUqgAAHhR9QEmje4vwZkMoEKiADwUKrPjkEAQwfsTs dur-pregnancy dur-pregnancy-ns ai-drug-info ai-drug-info-ns


# 1. 임산부 금기 여부 Retrieval

In [13]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

def dur_pregnacy_similarity(query):
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

    dur_vector_db = PineconeVectorStore(
        embedding=embeddings,
        index_name=PINECONE_INDEX_NAME,
        namespace=PINECONE_NAMESPACE,
        pinecone_api_key=PINECONE_API_KEY
    )
    
    results = dur_vector_db.similarity_search(    
    query=query,
    k=5,
    namespace=PINECONE_NAMESPACE)
    
    return {
        "medicine_name":query,
        "dur_context": "\n".join([doc.page_content for doc in results])
    }
    

In [27]:
dur_pregnacy_similarity('에이서캡슐(아세클로페낙)')

{'medicine_name': '에이서캡슐(아세클로페낙)',
 'dur_context': '성분명: aceclofenac\n제품명: 에이서캡슐(아세클로페낙)_(0.1g/1캡슐)\n업체명: 경동제약(주)\n금기등급: 2\n상세정보: 임부에 대한 안전성 미확립.임신 말기에 투여시 태아의 동맥관조기폐쇄 가능성.동물실험에서 비스테로이드성 소염진통제는 난산발생빈도 증가, 분만지연, 태아 생존율 감소 보고.임신 약 20주 이후 비스테로이드성 소염제의 사용은 태아의 신기능 이상을 일으켜 양수 과소증 유발 가능 및 경우에 따라 신생아 신장애 발생 가능\n성분명: cilostazol\n제품명: 시타졸서방캡슐100mg(실로스타졸)_(0.1g/1캡슐)\n업체명: 동광제약(주)\n금기등급: 2\n상세정보: 동물실험에서 이상태자의 증가 및 출생시의 저체중, 사망 증가 보고.\n성분명: cilostazol\n제품명: 시타졸서방캡슐200mg(실로스타졸)_(0.2g/1캡슐)\n업체명: 동광제약(주)\n금기등급: 2\n상세정보: 동물실험에서 이상태자의 증가 및 출생시의 저체중, 사망 증가 보고.\n성분명: ginkgo biloba leaf ext. + ginseng 30% ethanol ext.\n제품명: 징코산캡슐_(1캡슐)\n업체명: (주)한국파비스제약\n금기등급: 2\n상세정보: (정제, 액제, 캡슐제) 임신 중 투여에 대한 충분한 사용경험 적음 (주사제) -\n성분명: Diacerein\n제품명: 아세렌캡슐(디아세레인)_(50mg/1캡슐)\n업체명: 대화제약(주)\n금기등급: 2\n상세정보: 동물실험에서 태자 골형성 지연 보고'}

In [18]:
from langchain_core.runnables import RunnableLambda
chain = RunnableLambda(dur_pregnacy_similarity)
chain.invoke('에이서캡슐(아세클로페낙)')

{'medicine_name': '에이서캡슐(아세클로페낙)',
 'dur_context': '성분명: aceclofenac\n제품명: 에이서캡슐(아세클로페낙)_(0.1g/1캡슐)\n업체명: 경동제약(주)\n금기등급: 2\n상세정보: 임부에 대한 안전성 미확립.임신 말기에 투여시 태아의 동맥관조기폐쇄 가능성.동물실험에서 비스테로이드성 소염진통제는 난산발생빈도 증가, 분만지연, 태아 생존율 감소 보고.임신 약 20주 이후 비스테로이드성 소염제의 사용은 태아의 신기능 이상을 일으켜 양수 과소증 유발 가능 및 경우에 따라 신생아 신장애 발생 가능\n성분명: cilostazol\n제품명: 시타졸서방캡슐100mg(실로스타졸)_(0.1g/1캡슐)\n업체명: 동광제약(주)\n금기등급: 2\n상세정보: 동물실험에서 이상태자의 증가 및 출생시의 저체중, 사망 증가 보고.\n성분명: cilostazol\n제품명: 시타졸서방캡슐200mg(실로스타졸)_(0.2g/1캡슐)\n업체명: 동광제약(주)\n금기등급: 2\n상세정보: 동물실험에서 이상태자의 증가 및 출생시의 저체중, 사망 증가 보고.\n성분명: ginkgo biloba leaf ext. + ginseng 30% ethanol ext.\n제품명: 징코산캡슐_(1캡슐)\n업체명: (주)한국파비스제약\n금기등급: 2\n상세정보: (정제, 액제, 캡슐제) 임신 중 투여에 대한 충분한 사용경험 적음 (주사제) -\n성분명: Diacerein\n제품명: 아세렌캡슐(디아세레인)_(50mg/1캡슐)\n업체명: 대화제약(주)\n금기등급: 2\n상세정보: 동물실험에서 태자 골형성 지연 보고'}

# 2. 약 정보 Retrieval

In [21]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

def drug_info_similarity(query):
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

    dur_vector_db = PineconeVectorStore(
        embedding=embeddings,
        index_name=PINECONE_INDEX_NAME_2,
        namespace=PINECONE_NAMESPACE_2,
        pinecone_api_key=PINECONE_API_KEY
    )
    
    results = dur_vector_db.similarity_search(    
    query=query,
    k=5,
    namespace=PINECONE_NAMESPACE_2)
    
    return {
        "medicine_context": "\n".join([doc.page_content for doc in results])
    }

In [25]:
drug_info_similarity('에이서캡슐(아세클로페낙)')

{'medicine_context': '\ufeff업체명: (주)한국글로벌제약\n제품명: 뮤코틸캡슐(아세틸시스테인)\n품목기준코드: 201404707\n효능: 이 약은 급ㆍ만성기관지염, 기관지천식, 후두염, 부비동염, 낭성섬유증의 질환에서\xa0객담배출곤란에 사용합니다.\n복용방법: 급성질환에 성인은 1회 1캡슐(200 mg)씩, 1일 3회, 6~14세는 1회 1캡슐(200 mg)씩, 1일 2회 식전에 소량의 물과 함께 복용합니다.만성질환에 성인은 1회 1캡슐(200 mg)씩, 1일 2회, 6~14세는 1회\xa0\xa01/2\xa0캡슐(100 mg)씩, 1일 3회 식전에 소량의 물과 함께 복용합니다.낭성섬유증에 6세 이상은 1회 1캡슐(200 mg)씩, 1일 3회 식전에 소량의 물과 함께 복용합니다.\n알아야할 점: \n사용상 주의사항: 이 약에 과민증 환자,\xa0위•십이지장궤양,\xa02세 미만 영아, 갈락토오스 불내성, Lapp 유당분해효소 결핍증 또는 포도당-갈락토오스 흡수장애 등의 유전적인 문제가 있는 환자는 이 약을 복용하지 마십시오.이 약을 복용하기 전에\xa0임부 또는 임신하고 있을 가능성이 있는 여성 및 수유부는 의사 또는 약사와 상담하십시오.\n병용금지 약/음식: 경구용 항생제(아목시실린, 세푸록심, 독시사이클린, 에리트로마이신, 티암페니콜)와\xa0함께 복용 시 적어도 2시간\xa0간격을 두고 복용하십시오.\n이상반응 가능성: 드물게 구역, 구토, 구내염, 가슴쓰림, 설사, 두드러기, 가려움, 발진, 두통, 이명, 기관지경련(특히 천식 환자), 빈맥, 고혈압 등이 나타나는 경우\xa0즉각 복용을 중지하고\xa0의사 또는 약사와 상담하십시오.\n보관방법: 실온에서 보관하십시오.어린이의 손이 닿지 않는 곳에 보관하십시오.\n\ufeff업체명: 태극제약(주)\n제품명: 아로마에프캡슐\n품목기준코드: 200903224\n효능: 이 약은 치주치료 후 치은염, 경·중등도 치주염의 보조치료에 사용합니다.\n복용방법: 성인은 1회 1캡슐, 1일 3회 식

In [22]:
from langchain_core.runnables import RunnableLambda
chain = RunnableLambda(drug_info_similarity)
chain.invoke('에이서캡슐(아세클로페낙)')

{'medicine_context': '\ufeff업체명: (주)한국글로벌제약\n제품명: 뮤코틸캡슐(아세틸시스테인)\n품목기준코드: 201404707\n효능: 이 약은 급ㆍ만성기관지염, 기관지천식, 후두염, 부비동염, 낭성섬유증의 질환에서\xa0객담배출곤란에 사용합니다.\n복용방법: 급성질환에 성인은 1회 1캡슐(200 mg)씩, 1일 3회, 6~14세는 1회 1캡슐(200 mg)씩, 1일 2회 식전에 소량의 물과 함께 복용합니다.만성질환에 성인은 1회 1캡슐(200 mg)씩, 1일 2회, 6~14세는 1회\xa0\xa01/2\xa0캡슐(100 mg)씩, 1일 3회 식전에 소량의 물과 함께 복용합니다.낭성섬유증에 6세 이상은 1회 1캡슐(200 mg)씩, 1일 3회 식전에 소량의 물과 함께 복용합니다.\n알아야할 점: \n사용상 주의사항: 이 약에 과민증 환자,\xa0위•십이지장궤양,\xa02세 미만 영아, 갈락토오스 불내성, Lapp 유당분해효소 결핍증 또는 포도당-갈락토오스 흡수장애 등의 유전적인 문제가 있는 환자는 이 약을 복용하지 마십시오.이 약을 복용하기 전에\xa0임부 또는 임신하고 있을 가능성이 있는 여성 및 수유부는 의사 또는 약사와 상담하십시오.\n병용금지 약/음식: 경구용 항생제(아목시실린, 세푸록심, 독시사이클린, 에리트로마이신, 티암페니콜)와\xa0함께 복용 시 적어도 2시간\xa0간격을 두고 복용하십시오.\n이상반응 가능성: 드물게 구역, 구토, 구내염, 가슴쓰림, 설사, 두드러기, 가려움, 발진, 두통, 이명, 기관지경련(특히 천식 환자), 빈맥, 고혈압 등이 나타나는 경우\xa0즉각 복용을 중지하고\xa0의사 또는 약사와 상담하십시오.\n보관방법: 실온에서 보관하십시오.어린이의 손이 닿지 않는 곳에 보관하십시오.\n\ufeff업체명: 태극제약(주)\n제품명: 아로마에프캡슐\n품목기준코드: 200903224\n효능: 이 약은 치주치료 후 치은염, 경·중등도 치주염의 보조치료에 사용합니다.\n복용방법: 성인은 1회 1캡슐, 1일 3회 식

In [28]:
# 각 딕셔너리를 합쳐 하나의 딕셔너리 반환
def merge_dict(query):
    dict1 = dur_pregnacy_similarity(query)
    dict2 = drug_info_similarity(query)
    merged_dict = dict1 | dict2
    return merged_dict

In [40]:
example_dict = merge_dict('에이서캡슐(아세클로페낙)')

In [48]:
def generate_prompt(medicine_name, medicine_context, dur_context):
    return ChatPromptTemplate.from_messages([
        ('system', """
        Persona: You are a knowledgeable and experienced pharmacist with a passion to help people. 
        
        Role: As a pharmacist, your role is to provide simple and clear explanations about medicines for general people.
        
        Examples:
        Your explanation should cover the following:
        
        1. A brief explanation of the medicine, its effectiveness, dosage, precautions, side effects, and storage information.
        2. If the medicine is safe for pregnant women, including any contraindications, warning levels, and what they mean.
        
        Use the context provided below to answer the user's query.
        """),
        ('user', f"""
        Use the following pieces of retrieved context to answer the question. 
        If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
        
        Context about the medicine:
        {medicine_context}
        
        Context about the pregnancy-related contraindications:
        {dur_context}
        
        Medicine name: {medicine_name}
        """)
    ])

In [54]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import RunnableLambda, RunnableMap


def ai_pharmacist(question, merged_dict):
    prompt = generate_prompt(
    medicine_name=merged_dict['medicine_name'],
    medicine_context=merged_dict['medicine_context'],
    dur_context=merged_dict['dur_context']
    )
    
    model = ChatOpenAI(model='gpt-4o',temperature=1)
    
    # RunnableLambda를 사용하여 모델 호출
    pipeline = RunnableLambda(lambda x: model(prompt.format_messages({"user": x})))
    
    # 질문을 파이프라인에 전달
    result = pipeline.invoke(question)
    
    # output_parser = StrOutputParser()
    # chain = generate_prompt_runnable | model | output_parser
    # result = chain.invoke(inputs)
    # 
    return result
    

ImportError: cannot import name 'RunnableLambda' from 'langchain.chains' (C:\Users\playdata2\anaconda3\envs\dl_nlp2_env\Lib\site-packages\langchain\chains\__init__.py)

In [41]:
example_question = '이 약 임산부가 먹어도 되는지와 복용 시 주의사항 알려줘'

In [52]:
ai_pharmacist(example_question, example_dict)

TypeError: 'builtin_function_or_method' object is not subscriptable

In [34]:
from langchain_core.runnables import RunnablePassthrough
query = '타이레놀'
chain = {'question':RunnablePassthrough(), }

AttributeError: 'str' object has no attribute 'items'